In [8]:
import numpy as np
import pandas as pd

In [9]:
%run Problems.ipynb

In [20]:
def split_solution(solution):
    sublists = []
    sublist = []
    
    for item in solution:
        if item == (0,0):
            sublists.append(sublist)
            sublist = []
        else:
            sublist.append(item)
    
    sublists.append(sublist)
    
    return sublists


def combine_solution(trips):
    combined_solution = []
    for trip in trips:
        combined_solution.extend(trip)
        combined_solution.append((0,0))
    
    if combined_solution and combined_solution[-1] == (0,0):
        combined_solution.pop()
    
    return combined_solution


In [21]:
a = [(0,0),(0,0),(1, 0), (2, 1), (3, 0), (0, 0), (5, 1), (6, 0), (7, 1), (8, 0), (0, 0)]


In [22]:
s = split_solution(a)
c = combine_solution(s)

In [24]:
c

[(0, 0),
 (0, 0),
 (1, 0),
 (2, 1),
 (3, 0),
 (0, 0),
 (5, 1),
 (6, 0),
 (7, 1),
 (8, 0),
 (0, 0)]

In [5]:

#Vessel allocation helper functions

def min_range_needed(trip, preproseccing):
    """Function to find the lower cap for the passanger capacity for allocation vessels

    Args:
        trip (_type_): trip
        preproseccing (_type_): Preproseccing matrices

    Returns:
        biggest_move (float): The biggest jump between two nodes in the trip
    """
        

    distances_w_charging = preproseccing["DistancesWithCharging"]
    
    biggest_move = 0
    
    for i in range(len(trip)-1):
        p1 = trip[i][0]
        p2 = trip[i+1][0]
        dist = distances_w_charging[p1,p2]
        if dist>biggest_move:
            biggest_move = dist
            
    return biggest_move

def maximum_passengers(solution,problem):
    """This function goes through the trips in the solution, keeping track over how many passangers are in the vessel at each point. Then returns two lists:
            -A list of tuples (node_id, passangers onboard)
            -A list with the maximum number of passanger onboard during a trip.

    Args:
        solution (_type_): Generated solution by solver
        problem (_type_): A problem file containg inforamtion about the problem.
        
        
    ###
    PSEUDOCODE:
    
    Initialize a dictonary containg information about the jobs. (jobs = problem["Jobs"])
    Initialize a list to keep track of nodes visited (delivered_to_OTW).
    Initialize a list to keep track of the maximum number of passangers onboard for each trip
    
    Separate the solution into trips. Replace zeros with ','.
    
    Loop through the trips:
        
        Initialize a current capacity variable (onboard = 0) #Might not be relevant
        Initialize a list to keep track of change in capacity at each node (cap_list)
        
        Loop through each node in the trip (for i in len(trip))
            node = trip[i]
            If node == 0: (Maybe)
                contiunue
            n_crew, priority = jobs[node]
            At each node I can find out if I deliver crew (if node not in delivered_to_OTW)
            
            If not delivered: 
                cap_list[i] = -n_crew
            else:
                cap_list[i] = n_crew
        
        Using the cap_list, find the sum of all the negative numbers and insert this at the start of the list.
        Then going through the cap_list again and adding cumulatively. (n+1 = n + caplist[n+1])    
    
    """
    
    
    
    
    max_passangers = []
    
    trips = split_solution(solution)
    
    
    
    for t in range(len(trips)-1):
        trip = trips[t]
        max_passangers.append(single_max_pass(trip, problem))
                
    return max_passangers



def single_max_pass(trip, problem):
    
    jobs = problem["Jobs"]
    
    trip = [(0,0)] + trip
            
    cap_list = [0] * len(trip)
    
    for i in range(len(trip)):
        element = trip[i]
        
        
    
        if element == (0,0):
            continue
        node,binary = element
    
        n_crew = jobs[node]
        
        if binary == 0:
            cap_list[i] = -n_crew
        elif binary == 1:
            cap_list[i] = n_crew
    
    cap_list[0] = abs(sum(n for n in cap_list if n<0))

    for i in range(len(cap_list)-1):
        curr = cap_list[i+1]
        cap_list[i+1] = cap_list[i] + curr
    
    return max(cap_list)



In [3]:

def allocate_single_vessel(trip, problem):
    vessels = problem["Vessels"]
    preproseccing = problem["PreProseccing"]
    
    min_range = min_range_needed(trip, preproseccing)
    max_pass_cap = single_max_pass(trip, problem)
    if len(trip) == 0:
        return 0,True
    
    if max_pass_cap == 0:
    
        return 0,True
    
    useable_vessels = [vessel for vessel in vessels.values() if vessel[1] >= min_range and vessel[4] >= max_pass_cap]
    
    if len(useable_vessels) == 0:
        return 0,False  # infeasible
    else:
        chosen_vessel = min(useable_vessels, key=lambda x: x[2])
        return chosen_vessel[0],True
    
    

In [4]:
def allocate_vessels(solution,problem):
    """This function allocates a vessel to each trip in the solution.
    

    Args:
        problem (_type_): _description_
        solution (_type_): _description_
        preproseccing (_type_): _description_

    Returns:
        allocated_vessels (list): Returns a list which gives the chosen vessel for each trip
    """
    feasible = True
    allocated_vessels = []
    
    trips = split_solution(solution)
    
    
    
    for i in range(len(trips)-1):
        
        chosen_vessel, feasible_allocation = allocate_single_vessel(trips[i],problem)
        if not feasible_allocation:
            feasible = feasible_allocation
        allocated_vessels.append(chosen_vessel)    
    
    return (solution, allocated_vessels),feasible
    

In [3]:
#Charging helper functions
def too_far(distance_matrix, cs, p2,battery_range):
    return distance_matrix[cs,p2]>battery_range

def choose_cs(preproseccing,p1,p2,charging_stations,curr_range):
    preferred_cs=0
    
    charging_matrix = preproseccing["PreferredCS"]
    distance_w_charging_matrix = preproseccing["DistancesWithCharging"]
    
    
    cs_ids = [cs[0] for cs in charging_stations] 
    if p1 in cs_ids:
        preferred_cs = p1
    elif p2 == 0:
        preferred_cs, _ = charging_matrix[p1,p1]
    else:
        preferred_cs,_ = charging_matrix[p1,p2]
        
        if distance_w_charging_matrix[p1,preferred_cs]>curr_range:
            preferred_cs,_ = charging_matrix[p1,p1]

    return preferred_cs

In [ ]:

def add_charging_loop(trip,vessel,preproseccing,problem):
    
    charging_satations = problem["ChargingStations"]
    vessel_types = problem["Vessels"]
    
    _,battery_range,_,_,_,_ = vessel_types[vessel]
    distances_w_charging = preproseccing["DistancesWithCharging"]
    
    
    tour = [x[0] for x in trip]
    
    cum_range = get_cum_range(preproseccing, tour, battery_range)
    
    charging_done = []
    feasible = True
    
    for i in range(len(trip)-1):
        node1 = trip[i]
        node2 = trip[i+1]
        
        p1 = tour[i]
        p2 = tour[i+1]
        
        curr_range = cum_range[i]
        
        if curr_range < distances_w_charging[p1,p2]:
            preffered_cs = choose_cs(preproseccing,p1,p2,charging_satations,curr_range)
            
            trip, cum_range = new_tour(trip, node1, node2, preffered_cs, cum_range, preproseccing)
            charge_needed = abs(cum_range[-1]) +0.1
            charge_to_add = min(charge_needed, battery_range - cum_range[i+1])
            cum_range[i+1:] = [x + charge_to_add for x in cum_range[i+1:]]
            if cum_range[i+2] < 0:
                feasible = False
                break
            charging_done.append((preffered_cs,charge_to_add))
            
    return trip, charging_done,feasible

In [13]:
def charge_all_vessels(solution, problem):
    """This functio runs through the solution and adds charging to all the trips.

    Args:
        solution (_type_): _description_
        preproseccing (_type_): _description_
        problem (_type_): _description_

    Returns:
        _type_: returns the solution with added charging, the list of charging done (for the objective funciton) and a feasible boolean.
        
    #######################################################################    
    UPDATE!!!!
    
    Add it so that a tour which is outsourced does not go to charge!!!!!!
    
    
    I THINK THIS IS FIXED
    #######################################################################
    """
    
    preproseccing = problem["PreProseccing"]
    
    sol, vessels = solution
    
    trips = split_solution(sol)
    
    feasible = True
    chargelist = []
    
    for i in range(len(vessels)): 
       
       
        trip = trips[i]
        vessel = vessels[i]
        
        if vessel == 0:
            chargelist.append([])
            continue
        else:
            trip_with_padding =[(0,0)] + trip + [(0,0)]            
            
            trip_with_padding,charging_done,feasible_trip = add_charging_loop(trip_with_padding,vessel,preproseccing,problem)
            trips[i] = trip_with_padding[1:-1]
            chargelist.append(charging_done)
            feasible = feasible_trip
        
    
    sol = combine_solution(trips) #Combines the list of trips into a single array
    new_solution = sol,vessels, chargelist
    return new_solution,feasible
        

In [1]:
def postprocessing(solution,problem):
    """This function allocates vessels and adds charging to the given solution. It returns the original solution, the processed solution with allocation and charging and the feasibility of the charge.

    Args:
        solution (_type_): _description_
        problem (_type_): _description_

    Returns:
        solution _type_: The unmodified solution
        post_p_sol _type_: The solution with allocation and charging
        feasible_charge _type_: A boolean value indicating if the charge and allocation was feasible
    """
    
    allocated_sol,feasible_allocation = allocate_vessels(solution,problem)
    post_p_sol,feasible_charge = charge_all_vessels(allocated_sol,problem)
    
    return solution, post_p_sol, (feasible_charge and feasible_allocation) 